# Preamble

In [ ]:
import os
import numpy as np                                       # fast vectors and matrices
import matplotlib.pyplot as plt                          # plotting
from scipy import fft                                    # fast fourier transform

from intervaltree import Interval,IntervalTree

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

%matplotlib inline

In [ ]:
fs = 44100            # samples/second
window_size = 2048    # fourier window size
d = 500               # number of features
m = 128               # number of distinct notes
n = 1000              # training data points per recording

musicnet = os.environ['MUSICNET']

In [ ]:
data = np.load(open(musicnet,'rb'))

# split our dataset into train and test
test_data = ['2303','2382','1819']
train_data = [f for f in data.files if f not in test_data]

In [ ]:
# ReLUgrams
filters = np.empty((2*d,window_size))
x = np.linspace(0, 2*np.pi, window_size, endpoint=False)
for k in range(d):
    filters[k] = np.cos(k*x)
    filters[d+k] = np.sin(k*x)
    
def featurize(X):
    return np.log(1 + np.abs(np.dot(X,filters[:d].T)) + np.abs(np.dot(X,filters[d:].T)))

In [ ]:
# create the test set
Xtest = np.empty([3*7500,d])
Ytest = np.zeros([3*7500,m])
for i in range(len(test_data)):
    X,Y = data[test_data[i]]
    for j in range(7500):
        s = fs+j*512 # start from one second to give us some wiggle room for larger segments
        norm = np.linalg.norm(X[s:s+window_size]) + 10e-6
        #Xtest[7500*i + j] = np.log(1 + np.abs(fft(X[s:s+window_size]/norm))[0:d])
        Xtest[7500*i + j] = featurize(X[s:s+window_size]/norm)
        
        # label stuff that's on in the center of the window
        for label in Y[s+window_size/2]:
            Ytest[7500*i + j,label.data[1]] = 1

# Linear Model

In [ ]:
# sufficient statistics for least squares
XTX = np.zeros((d,d))
XTY = np.zeros((d,m))

# Warning: this could take some time
Xs = np.empty((n,d))
for recording in train_data:
    print recording, ',',
    X,Y = data[recording]
    s = np.random.randint(window_size/2,len(X)-window_size/2,n)
    Ys = np.zeros((n,m))
    for i in range(n):
        norm = np.linalg.norm(X[s[i]-window_size/2:s[i]+window_size/2]) + 10e-6
        #Xs[i] = np.log(1 + np.abs(fft(X[s[i]-window_size/2:s[i]+window_size/2]/norm))[0:d])
        Xs[i] = featurize(X[s[i]-window_size/2:s[i]+window_size/2]/norm)
        for label in Y[s[i]]:
            Ys[i,label.data[1]] = 1
    XTX += (1./n)*np.dot(Xs.T,Xs)
    XTY += (1./n)*np.dot(Xs.T,Ys)
XTX /= float(len(train_data))
XTY /= float(len(train_data))

In [ ]:
grid = [2**i for i in range(-10,0)]
average_precision = []
for r in grid:
    print r,', ',
    w = np.linalg.solve(XTX + r*np.eye(XTX.shape[0]),XTY)
    
    Yhat = np.dot(Xtest,w)
    yflat = Ytest.reshape(Ytest.shape[0]*Ytest.shape[1])
    yhatflat = Yhat.reshape(Yhat.shape[0]*Yhat.shape[1])
    average_precision.append(average_precision_score(yflat, yhatflat))
    
fig = plt.figure()
plt.plot(range(-10,-0),average_precision,color=(41/255.,104/255.,168/255.),linewidth=3)
fig.axes[0].set_xlabel('regularizer (order of magnitude)')
fig.axes[0].set_ylabel('average precision')

In [ ]:
w = np.linalg.solve(XTX + 10e-6*np.eye(XTX.shape[0]),XTY)
Yhat = np.dot(Xtest,w)
yflat = Ytest.reshape(Ytest.shape[0]*Ytest.shape[1])
yhatflat = Yhat.reshape(Yhat.shape[0]*Yhat.shape[1])
precision, recall, _ = precision_recall_curve(yflat, yhatflat)

fig = plt.figure()
plt.plot(recall,precision,color=(41/255.,104/255.,168/255.),linewidth=3)
fig.axes[0].set_xlabel('recall')
fig.axes[0].set_ylabel('precision')

# MIREX evaluation

In [ ]:
import mir_eval

In [ ]:
def estimate(X,subdiv=50):
    return np.dot(X,w)

In [ ]:
Xvalidation = np.zeros([50*len(train_data),d])
Yvalidation = np.zeros([50*len(train_data),m])
for i in range(len(train_data)):
    recording = train_data[i]
    print recording, ',',
    X,Y = data[recording]
    # 50 random samples from each recording
    s = np.random.randint(window_size/2,len(X)-window_size/2,50)
    for j in range(50):
        norm = np.linalg.norm(X[s[j]-window_size/2:s[j]+window_size/2]) + 10e-6
        #Xvalidation[50*i+j] = np.log(1 + np.abs(fft(X[s[j]-window_size/2:s[j]+window_size/2]/norm))[0:d])
        Xvalidation[50*i+j] = featurize(X[s[j]-window_size/2:s[j]+window_size/2])
        
        # label stuff that's on in the center of the window
        for label in Y[s[j]]:
            Yvalidation[50*i+j,label.data[1]] = 1

In [ ]:
Yhatbase = estimate(Xvalidation)

# single threshold
density = 500
P = np.empty(density)
R = np.empty(density)
F = np.empty(density)
for i in np.arange(density):
    if i % 100 == 0: print '.',
    c = i/float(density)
    Yhat = Yhatbase>c
    true_positives = np.sum(Yhat*Yvalidation)
    P[i] = true_positives/np.sum(Yhat)
    R[i] = true_positives/np.sum(Yvalidation)
    F[i] = 2*(P[i]*R[i])/(P[i]+R[i])

plt.plot(F)
i = np.argmax(F)
c = i/float(density)
print c

In [ ]:
Yhatbase = estimate(Xtest)

Yhat = Yhatbase>c
Yhatlist = []
Ytestlist = []
for i in range(len(Yhat)):
    fhat = []
    ftest = []
    for note in range(128):
        if Yhat[i][note] == 1:
            fhat.append(440.*2**((note - 69.)/12.))
        if Ytest[i][note] == 1:
            ftest.append(440.*2**((note - 69.)/12.))
    Yhatlist.append(np.array(fhat))
    Ytestlist.append(np.array(ftest))

In [ ]:
P,R,Acc,Esub,Emiss,Efa,Etot,cP,cR,cAcc,cEsub,cEmiss,cEfa,cEtot = \
mir_eval.multipitch.metrics(np.arange(len(Ytestlist))/100.,Ytestlist,np.arange(len(Yhatlist))/100.,Yhatlist)

print P
print R
print Acc
print Etot
print Esub
print Emiss
print Efa

print '-----'

print cP
print cR
print cAcc
print cEtot
print cEsub
print cEmiss
print cEfa

# Precision/Recall

In [ ]:
Yhattestbase = estimate(Xtest)
Yhat = Yhattestbase>c
true_positives = np.sum(Yhat*Ytest)
P = true_positives/(np.sum(Yhat))
R = true_positives/(np.sum(Ytest))
F = 2*(P*R)/(P+R)
print P
print R
print F

In [ ]:
yflat = Ytest.reshape(Ytest.shape[0]*Ytest.shape[1])
yhatflat = Yhattestbase.reshape(Yhattestbase.shape[0]*Yhattestbase.shape[1])

precision, recall, _ = precision_recall_curve(yflat, yhatflat)
ap = average_precision_score(yflat, yhatflat)
plt.plot(recall,precision)
print ap

In [ ]:
plt.rcParams.update({'font.size': 12})

fig = plt.figure()
plt.plot(recall,precision)
fig.axes[0].set_xlabel('recall')
fig.axes[0].set_ylabel('precision')

plt.tight_layout()
plt.savefig('linear_pr.eps',format='eps', dpi=1000)